In [1]:
import numpy as np
import pandas as pd

## Values considered "missing"

As data comes in many shapes and forms, pandas aims to be flexible with regard to handling missing data. While NaN is the default missing value marker for reasons of computational speed and convenience, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object. In many cases, however, the Python None will arise and we wish to also consider that “missing” or “not available” or “NA”.

+ If you want to consider inf and -inf to be “NA” in computations, you can set pandas.options.mode.use_inf_as_na = True.

In [2]:
df = pd.DataFrame(np.random.randn(5,3),index=['a','c','e','f','h'],columns=['one','two','three'])
df['four']='bar'
df['five']=df['one']>0
df

,one,two,three,four,five
a,1.021775,0.695072,1.052658,bar,True
c,-0.215951,0.957422,0.374447,bar,False
e,-0.073123,-0.476798,1.016094,bar,False
f,-0.441731,0.053236,-1.044367,bar,False
h,1.181962,-0.499605,-0.175427,bar,True


In [3]:
df2=df.reindex(['a','b','c','d','e','f','g','h'])
df2

,one,two,three,four,five
a,1.021775,0.695072,1.052658,bar,True
b,NaN,NaN,NaN,NaN,NaN
c,-0.215951,0.957422,0.374447,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-0.073123,-0.476798,1.016094,bar,False
f,-0.441731,0.053236,-1.044367,bar,False
g,NaN,NaN,NaN,NaN,NaN
h,1.181962,-0.499605,-0.175427,bar,True


+ To make detecting missing values easier (and across different array dtypes), pandas provides the isna() and notna() functions, which are also methods on Series and DataFrame objects:

In [4]:
pd.isna(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [5]:
df2['four'].notna()

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: four, dtype: bool

In [6]:
df2.isna()

,one,two,three,four,five
a,False,False,False,False,False
b,True,True,True,True,True
c,False,False,False,False,False
d,True,True,True,True,True
e,False,False,False,False,False
f,False,False,False,False,False
g,True,True,True,True,True
h,False,False,False,False,False


+ One has to be mindful that in Python (and NumPy), the nan's don’t compare equal, but None's do. Note that pandas/NumPy uses the fact that np.nan != np.nan, and treats None like np.nan.

In [7]:
None==None

True

In [8]:
np.nan==np.nan

False

## Integer dtypes and missing data

+ Because NaN is a float, a column of integers with even one missing values is cast to floating-point dtype.
+ Pandas provides a nullable integer array, which can be used by explicitly requesting the dtype:

In [11]:
pd.Series([1,2,np.nan,4],dtype=pd.Int64Dtype())

0      1
1      2
2    NaN
3      4
dtype: Int64

## Datetimes

+ For datetime64[ns] types, NaT represents missing values. 
+ pandas objects provide compatibility between NaT and NaN.

In [12]:
df2=df.copy()
df2

,one,two,three,four,five
a,1.021775,0.695072,1.052658,bar,True
c,-0.215951,0.957422,0.374447,bar,False
e,-0.073123,-0.476798,1.016094,bar,False
f,-0.441731,0.053236,-1.044367,bar,False
h,1.181962,-0.499605,-0.175427,bar,True


In [13]:
df2['timestamp']=pd.Timestamp('20200101')
df2

,one,two,three,four,five,timestamp
a,1.021775,0.695072,1.052658,bar,True,2020-01-01
c,-0.215951,0.957422,0.374447,bar,False,2020-01-01
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,1.181962,-0.499605,-0.175427,bar,True,2020-01-01


In [14]:
df2.loc[['a','c','h'],['one','timestamp']]=np.nan
df2

,one,two,three,four,five,timestamp
a,NaN,0.695072,1.052658,bar,True,NaT
c,NaN,0.957422,0.374447,bar,False,NaT
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,NaN,-0.499605,-0.175427,bar,True,NaT


In [15]:
df2.dtypes.value_counts()

float64           3
object            1
bool              1
datetime64[ns]    1
dtype: int64

## Inserting missing values

You can insert missing values by simply assigning to containers. The actual missing value used will be chosen based on the dtype.

For example, numeric containers will always use NaN regardless of the missing value type chosen:

In [16]:
s = pd.Series([1,2,3])
s[0]=None
s

0    NaN
1    2.0
2    3.0
dtype: float64

Likewise, datetime containers will always use NaT.

For object containers, pandas will use the value given:

In [17]:
s = pd.Series(['a','b','c'])
s[0]=None
s[1]=np.nan
s

0    None
1     NaN
2       c
dtype: object

## Calculations with missing data

Missing values propagate naturally through arithmetic operations between pandas objects.

In [18]:
a = pd.DataFrame(np.random.randn(5,2),index=['a','c','e','f','h'],columns=['one','two'])
a.loc[['a','c'],['one']]=np.nan
a

,one,two
a,NaN,-0.441133
c,NaN,0.851738
e,-1.114486,0.329013
f,1.409369,0.814200
h,-1.130500,0.277409


In [19]:
b = pd.DataFrame(np.random.randn(5,3),index=['a','c','e','f','h'],columns=['one','two','three'])
b.loc[['a','c','h'],['one']]=np.nan
b

,one,two,three
a,NaN,-0.015943,-2.070278
c,NaN,-0.330057,-0.325387
e,0.675818,-1.672028,-0.294238
f,-1.670087,1.568461,0.158406
h,NaN,-0.699025,0.753526


In [20]:
a+b

,one,three,two
a,NaN,NaN,-0.457076
c,NaN,NaN,0.521681
e,-0.438668,NaN,-1.343015
f,-0.260718,NaN,2.382661
h,NaN,NaN,-0.421616


+ When summing data, NA (missing) values will be treated as zero.
+ If the data are all NA, the result will be 0.
+ Cumulative methods like cumsum() and cumprod() ignore NA values by default, but preserve them in the resulting arrays. To override this behaviour and include NA values, use skipna=False.

In [21]:
df2

,one,two,three,four,five,timestamp
a,NaN,0.695072,1.052658,bar,True,NaT
c,NaN,0.957422,0.374447,bar,False,NaT
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,NaN,-0.499605,-0.175427,bar,True,NaT


In [22]:
df2['one'].sum()

-0.5148535701687299

In [23]:
df2['one'].cumsum()

a         NaN
c         NaN
e   -0.073123
f   -0.514854
h         NaN
Name: one, dtype: float64

In [24]:
df2['one'].cumsum(skipna=False)

a   NaN
c   NaN
e   NaN
f   NaN
h   NaN
Name: one, dtype: float64

In [25]:
df2.mean(1)

a    0.915910
c    0.443956
e    0.116543
f   -0.358216
h    0.108323
dtype: float64

## Sum/prod of empties/nans

+ The sum of an empty or all-NA Series or column of a DataFrame is 0.

In [26]:
pd.Series([np.nan]).sum()

0.0

In [27]:
pd.Series([],dtype='float64').sum()

0.0

+ The product of an empty or all-NA Series or column of a DataFrame is 1.

In [28]:
pd.Series([np.nan]).prod()

1.0

In [29]:
pd.Series([],dtype='float64').prod()

1.0

## NA values in GroupBy

+ NA groups in GroupBy are automatically excluded. 

In [34]:
df2.groupby('one').mean()

,two,three,five
one,,,
-0.441731,0.053236,-1.044367,False
-0.073123,-0.476798,1.016094,False


## Filling missing values : fillna

+ fillna() can “fill in” NA values with non-NA data in a couple of ways.

In [36]:
#Replace NA with a scalr value
df2.fillna(0)

,one,two,three,four,five,timestamp
a,0.000000,0.695072,1.052658,bar,True,0
c,0.000000,0.957422,0.374447,bar,False,0
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01 00:00:00
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01 00:00:00
h,0.000000,-0.499605,-0.175427,bar,True,0


In [37]:
df2['one'].fillna('missing')

a      missing
c      missing
e   -0.0731229
f    -0.441731
h      missing
Name: one, dtype: object

In [42]:
#Fill gaps forward or backward
df2.fillna(method='pad') #'pad' propagates last valid observation forward to next valid

,one,two,three,four,five,timestamp
a,NaN,0.695072,1.052658,bar,True,NaT
c,NaN,0.957422,0.374447,bar,False,NaT
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,-0.441731,-0.499605,-0.175427,bar,True,2020-01-01


+ If we only want consecutive gaps filled up to a certain number of data points, we can use the limit keyword:

In [44]:
df2.fillna(method='pad',limit=1)

,one,two,three,four,five,timestamp
a,NaN,0.695072,1.052658,bar,True,NaT
c,NaN,0.957422,0.374447,bar,False,NaT
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,-0.441731,-0.499605,-0.175427,bar,True,2020-01-01


+ pad/ffill : fills values forward
+ bfill/backfill : fills values bakward
+ With time series data, using pad/ffill is extremely common so that the “last known value” is available at every time point.

+ You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series must match the columns of the frame you wish to fill. The use case of this is to fill a DataFrame with the mean of that column.

## Dropping axis labels with missing data : dropna()

+ You may wish to simply exclude labels from a data set which refer to missing data. To do this, use dropna():

In [46]:
df2

,one,two,three,four,five,timestamp
a,NaN,0.695072,1.052658,bar,True,NaT
c,NaN,0.957422,0.374447,bar,False,NaT
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01
h,NaN,-0.499605,-0.175427,bar,True,NaT


In [47]:
df2.dropna(axis=0)

,one,two,three,four,five,timestamp
e,-0.073123,-0.476798,1.016094,bar,False,2020-01-01
f,-0.441731,0.053236,-1.044367,bar,False,2020-01-01


In [48]:
df2.dropna(axis=1)

,two,three,four,five
a,0.695072,1.052658,bar,True
c,0.957422,0.374447,bar,False
e,-0.476798,1.016094,bar,False
f,0.053236,-1.044367,bar,False
h,-0.499605,-0.175427,bar,True


In [49]:
df2['one'].dropna()

e   -0.073123
f   -0.441731
Name: one, dtype: float64

## Replacing generic values

Often times we want to replace arbitrary values with other values.

replace() in Series and replace() in DataFrame provides an efficient yet flexible way to perform such replacements.

For a Series, you can replace a single value or a list of values by another value:

In [55]:
s=pd.Series([-1,1,2,3,4])
s

0   -1
1    1
2    2
3    3
4    4
dtype: int64

In [57]:
s.replace(-1,5)

0    5
1    1
2    2
3    3
4    4
dtype: int64

You can replace a list of values by a list of other values:

In [58]:
s.replace([4,2,3],[10,9,8])

0    -1
1     1
2     9
3     8
4    10
dtype: int64

You can also specify a mapping dict:

In [60]:
s.replace({-1:100,1:1000})

0     100
1    1000
2       2
3       3
4       4
dtype: int64

For a DataFrame, you can specify individual values by column:

In [61]:
df = pd.DataFrame({'a':[0,1,2,3,4],'b':[5,6,7,8,9]})
df

,a,b
0,0,5
1,1,6
2,2,7
3,3,8
4,4,9


In [62]:
df.replace({'a':0,'b':5},100)

,a,b
0,100,100
1,1,6
2,2,7
3,3,8
4,4,9
